In [1]:
import numpy as np
import itertools

In [2]:
def heaviside(x):
    return np.array(1 * (x >= 0))

In [22]:
def extension_bool(p,V):
    """
    Compute a boolean vector that represents the extension of p in V
    
    Inputs:
        p - a vector of shape (M,) with elements from {-1,0,1}.  The partially specified feature vector
        V-  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        extension - a vector of shape (L) with elements from {1,0}.  extension[l]=1 iff V[l,:] is in the extension of p
    """
    
    E = np.dot(V,p)/np.sum(abs(p))
    return heaviside(E-1)
def extension_mat(p,V):
    """
    Returns a matrix representing the extension of p in V
    
    Inputs:
        p - a vector of shape (M,) with elements from {-1,0,1}.  The partially specified feature vector
        V-  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        A - A matrix of shape (N,M) with elements from {-1,1}, where N <=L, and the rows of A are the rows of V in the
            extension of p
    """
    
    bool_vec = extension_bool(p,V)
    return V[bool_vec>0,:]
def check_equivalent(p,q,V):
    """
    Checks if p and q have the same extension in V
    
    Inputs:
        p,q - vectors of shape (M,) with elements from {-1,0,1}.  The partially specified feature vectors
        V-  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        match - bool, whether or not p and q have the same extension 
    """
    p_bool = extension_bool(p,V)
    q_bool = extension_bool(q,V)
    M = np.shape(q_bool)[0]
    return all([p_bool[i]==q_bool[i] for i in range(M)])
def equivalence_class(p,V):
    """
    Given a set of feature vectors V and a query partially specified feature vector p, return all partially
    specified vectors q that have the same extension in V as p.
    
    Inputs:
        p - a vector of shape (M,) with elements from {-1,0,1}.  The partially specified feature vector
        V-  a matrix of shape (L,M) with elements from {-1,1}.  The feature vectors
    Outputs:
        R - a list of vectors where R[i] has shape (M,) and all vectors in R have the same extension in V as p
    """
    
    M = np.shape(p)[0]
    L = np.shape(V)[0]
    V_sums = np.sum(V,axis=0)
    bool_p = extension_bool(p,V)
    
    A = V[bool_p>0,:]#shape is (N,M)
    N = np.shape(A)[0]
    A_sums = np.sum(A,axis=0)
    
    allowed_specifications = []
    prespecified_indices = []
    for i in range(M):
        if p[i]==0:
            #we can flip this position to a signed value if only if all A[:,i] or equivalently A_sums[i]== +/- N
            if A_sums[i]==N:
                allowed_specifications.append((i,1)) #they're all 1 so we can mutate p[i] to 1
            elif A_sums[i]==-N:
                allowed_specifications.append((i,-1)) #they're all -1 so we can mutate p[i] to -1
        else:
            prespecified_indices.append(i)
    
    #which indices of p can be zeroed out without changing the extension of p in V?
    
    T = L - N
    B = V[bool_p<=0,:]#shape is (L-N,M), p is not compatible with any vector in B
    
    #now we must check which subsets of the specified indices can be zeroed out without making the resulting 
    # 'de-specified' vector consistent any vectors in B
    
    return allowed_specifications

In [19]:
p = np.array([-1,0,1])
V = np.array([
    [1,1,1],
    [1,1,1],
    [1,-1,1]
])
extension_mat(p,V).ndim

2

In [13]:
test = np.array([1,0,1,0,1])
B[test>0,:]

array([[0, 1],
       [4, 5],
       [8, 9]])

In [ ]:
"""
If a vector is in V but not in A, that means it differs from p in at least one of the places p has an assigned value

If flipping a 1 to 0 in p would add a vector from V to A that wasn't there before, then that new vector agreed with p
everywhere but at the flipped index
"""